<a href="https://colab.research.google.com/github/TheNeodev/Retrieval-based-Voice-Conversion-WebUI/blob/main/Retrieval_based_Voice_Conversion_WebUI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Retrieval-based-Voice-Conversion-WebUI**

madew.svg
licr.svg

cutecounter.svg

An easy-to-use voice conversion framework based on VITS.

In [ ]:
#@title INSTALL RVC

from IPython.display import clear_output
from ipywidgets import Button
import subprocess, shlex, os
import codecs




mount_drive = False #@param {type:"boolean"}


if mount_drive:
    from google.colab import drive
    drive.mount('/content/drive')

var = "We"+"bU"+"I"
test = "Voice"
c_word = "Conversion"
r_word = "Retrieval"

!git clone https://github.com/RVC-Project/{r_word}-based-{test}-{c_word}-{var} /content/RVC

%cd RVC

!apt -y install -qq aria2
pretrains = ["f0D32k.pth","f0G32k.pth"]
new_pretrains = ["f0Ov2Super32kD.pth","f0Ov2Super32kG.pth"]

for file in pretrains:
    if not os.path.exists(f"/content/RVC/assets/pretrained_v2/{file}"):
        command = "aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/%s%s%s/resolve/main/pretrained_v2/%s -d /content/RVC/assets/pretrained_v2 -o %s" % ("Voice","Conversion","WebUI",file,file)
        try:
            subprocess.run(shlex.split(command))
        except Exception as e:
            print(e)

for file in new_pretrains:
    if not os.path.exists(f"/content/RVC/assets/pretrained_v2/{file}"):
        command = "aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/poiqazwsx/Ov2Super32kfix/resolve/main/%s -d /content/RVC/assets/pretrained_v2 -o %s" % (file,file)
        try:
            subprocess.run(shlex.split(command))
            print(shlex.split(command))
        except Exception as e:
            print(e)


!python3 -m pip install --upgrade pip==24.0

!pip install -r requirements/main.txt



if not os.path.exists("/content/sepenv"):
    !virtualenv sepenv

!source sepenv/bin/activate; pip install "audio-separator[gpu]"


if not "installed_yt" in locals():
    ! pip install -U yt-dlp -q
    ! wget -O - -q  https://github.com/yt-dlp/FFmpeg-Builds/releases/download/latest/ffmpeg-master-latest-linux64-gpl.tar.xz | xz -qdc| tar -x
    ! pip install pydub -q
    if not os.path.exists('/content/sepenv'):
        ! pip install virtualenv -q
        ! virtualenv sepenv
\
installed_yt = True


!wget -nc https://huggingface.co/Rejekts/project/resolve/main/download_files.py -O /content/RVC/download_files.py
!python download_files.py
Button(description="\u2714 Success", button_style="success")

In [ ]:
%cd /content
#@title Create dataset from YouTube video
youtube_url = "https://www.youtube.com/watch?v=P52rHqq-ugg" #@param {type:"string"}
separation_folder = "/content/separation_files"
dataset_folder = "/content/dataset"
os.makedirs(separation_folder,exist_ok=True)

import re, os
from yt_dlp import YoutubeDL
from IPython.display import clear_output, display, Audio
from ipywidgets import Button

def get_video_id(url):
    pattern = r"(?:v=|\/)([0-9A-Za-z_-]{11})"
    match = re.search(pattern, url)
    if match:
        return match.group(1)
    return None

video_id = get_video_id(youtube_url)
downloads_folder = "/content/downloads"
os.makedirs(downloads_folder,exist_ok=True)
video_url = f"https://youtu.be/{video_id}"
output_name = f"{video_id}.wav"
output_path = os.path.join(downloads_folder,output_name)
! yt-dlp -xv --ffmpeg-location ffmpeg-master-latest-linux64-gpl/bin --audio-format wav  -o $output_path -- {video_url}
if os.path.exists(dataset_folder):
    delete = input(f"Do you want to delete the existing {dataset_folder} folder? (y/n)")
    if delete.lower() == 'y':
        ! rm -r $dataset_folder
        ! mkdir -p $dataset_folder
    else:
        print("Keeping the existing dataset folder.")

def remove_parentheses(filename):
    # Get the directory and filename
    directory = os.path.dirname(filename)
    base_name = os.path.basename(filename)
    # Remove parentheses from the filename
    new_name = re.sub(r'[()]', '', base_name)
    # Join the new name with the directory
    new_path = os.path.join(directory, new_name)
    # Rename the file
    os.rename(filename, new_path)
    return new_path

# Remove the instrumental
! source sepenv/bin/activate; audio-separator $output_path --model_filename MDX23C-8KFFT-InstVoc_HQ_2.ckpt --output_dir {separation_folder}
# Fix filename
vocals_file = f"{separation_folder}/{video_id}_(Vocals)_MDX23C-8KFFT-InstVoc_HQ_2.flac"
remove_parentheses(vocals_file)
vocals_file = f"{separation_folder}/{video_id}_Vocals_MDX23C-8KFFT-InstVoc_HQ_2.flac"
# Remove the Reverb
! source sepenv/bin/activate; audio-separator {vocals_file} --model_filename UVR-DeEcho-DeReverb.pth --output_dir {dataset_folder}
! rm final/*\(Reverb\)*.flac
clear_output()
Button(description="\u2714 Done",button_style="success")

In [ ]:
#@title RUN RVC

import codecs

uipy = codecs.decode('jro.cl','rot_13')


!python $uipy --global_link